### Feet Anchoring

In [1]:
import time,pickle,mujoco
import numpy as np
import matplotlib.pyplot as plt
from mujoco_parser import MuJoCoParserClass
from util import rpy2r,r2quat,MultiSliderClass,animiate_motion_with_slider,pr2t,feet_anchoring
np.set_printoptions(precision=2,suppress=True,linewidth=100)
plt.rc('xtick',labelsize=6); plt.rc('ytick',labelsize=6)
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
print ("MuJoCo version:[%s]"%(mujoco.__version__))

MuJoCo version:[2.3.6]


### Initialize tick slider

In [2]:
# Open this once, and let's not close this. There's some problem with closing tkinter obj.
tick_slider = MultiSliderClass(
    n_slider=1,title='Tick slider',window_width=500,window_height=100,
    x_offset=950,y_offset=900,slider_width=350,label_texts=['Tick'],
    slider_mins=[0],slider_maxs=[100-1],slider_vals=[0],resolution=0.5,
    ADD_PLAYBACK=True,VERBOSE= False)
print ("[%s] Ready."%(tick_slider.title))

[Tick slider] Ready.


### Parse `scene_common_rig_v2.xml`

In [3]:
xml_path = '../asset/common_rig/scene_common_rig_v2.xml'
env = MuJoCoParserClass(name='Common Rig',rel_xml_path=xml_path,VERBOSE=False)
# Modify the color of body exluding 'world'
for body_name in env.body_names:
    if body_name in ['world']: continue
    body_idx = env.body_names.index(body_name)
    geom_idxs = [idx for idx,val in enumerate(env.model.geom_bodyid) if val==body_idx]
    for geom_idx in geom_idxs: env.model.geom(geom_idx).rgba = [0.9,0.9,0.9,0.5]
# Set which joints to control
rev_joint_names = env.ctrl_names 
print ("Done.")

Done.


### Load motion

In [4]:
# pkl_path = '../data/VAAI_DIRECT_09_02_a_M1.pkl'
pkl_path = '../data/VAAI_DIRECT_03_01_a_M1.pkl'
# pkl_path = '../data/VAAI_DIRECT_10_01_c_M1.pkl'
# pkl_path = '../data/VAAI_DIRECT_20_01_a_M1.pkl'
with open(pkl_path, 'rb') as f: data = pickle.load(f)
q_list         = data['qpos'][:,list(range(0,4))+list(range(5,12))+list(range(13,37))] # [L x 35]
quat_root_list = r2quat(data['root_rot']) # [L x 4]
p_root_list    = data['root_translation'] # [L x 3]
L = q_list.shape[0]
print ("L:[%d]"%(L))

L:[136]


In [5]:
# Raw motion
animiate_motion_with_slider(
    env,tick_slider,q_list,p_root_list,quat_root_list,rev_joint_names,
    PLAY_AT_START=True)

Pressed ESC
Quitting.


### Feet anchoring
Fix the locations of two feets on the ground

In [6]:
print ("Start feet anchoring...")
res = feet_anchoring(env,q_list,quat_root_list,p_root_list,rev_joint_names,
                     p_cfoot_offset = np.array([0,0,0.02]),d_rf2lf_custom=0.3,
                     ik_th=1e-3,ik_iters=5000,ANIMATE_IK=False)
print ("Feet anchoring done (L:[%d])"%(res['L']))
p_root_centered_list,q_feetanchor_list = res['p_root_centered_list'],res['q_feetanchor_list']

Start feet anchoring...
Feet anchoring done (L:[136])


### Animate motion

In [7]:
# Raw motion
animiate_motion_with_slider(
    env,tick_slider,q_list,p_root_list,quat_root_list,rev_joint_names,
    PLAY_AT_START=True)

2023-10-09 22:40:50.351 python[49572:2964599] IMKClient Stall detected, *please Report* your user scenario attaching a spindump (or sysdiagnose) that captures the problem - (imkxpc_bundleIdentifierWithReply:) block performed very slowly (369.44 secs).


Pressed ESC
Quitting.


In [8]:
# (intermediate) Centered motion
animiate_motion_with_slider(
    env,tick_slider,q_list,p_root_centered_list,quat_root_list,rev_joint_names,
    PLAY_AT_START=True)

Pressed ESC
Quitting.


In [9]:
# Feet-anchored motion
animiate_motion_with_slider(
    env,tick_slider,q_feetanchor_list,p_root_centered_list,quat_root_list,rev_joint_names,
    PLAY_AT_START=True)

Pressed ESC
Quitting.
